# 目录
## 1. 导入模块
## 2. tf.data.Dataset.from_tensor_slices
### 2.1 一维数组
### 2.2 二位数组
### 2.3 元组
### 2.4 字典
### 2.5 组合
  
## 3. shuffle的使用
## 4. repeat batch 的使用
## 5. take 从dataset取几个元素
## 6. interleave
## 7.  map的用法

## 1. 导入模块

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. tf.data.Dataset.from_tensor_slices
>该api的作用是，把内存的数据分装成`dataset`，并且`dataset`是一个迭代器；往里面传什么样的格式(元组，字典)数据，迭代的时候就返回什么样的格式

### 2.1 一维数组

In [2]:
dataset1 = tf.data.Dataset.from_tensor_slices(np.arange(10))
print(dataset1)

<TensorSliceDataset shapes: (), types: tf.int64>


In [3]:
for item in dataset1:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


### 2.2 二维数组

In [4]:
dataset1 = tf.data.Dataset.from_tensor_slices(np.random.normal(size=(8, 3)))
for item in dataset1:
    print(item)                                

tf.Tensor([-1.48846764 -0.88762542 -1.20111511], shape=(3,), dtype=float64)
tf.Tensor([-0.10914715 -1.69061133 -1.19204482], shape=(3,), dtype=float64)
tf.Tensor([ 0.02946257 -0.56475841 -0.78798879], shape=(3,), dtype=float64)
tf.Tensor([ 0.80156687 -0.38138432 -0.21904758], shape=(3,), dtype=float64)
tf.Tensor([ 0.70102761 -0.00352738  0.43777427], shape=(3,), dtype=float64)
tf.Tensor([-0.28952003 -1.8086101   2.18609337], shape=(3,), dtype=float64)
tf.Tensor([-0.03285051  0.4882148   1.08820751], shape=(3,), dtype=float64)
tf.Tensor([ 0.30932759 -0.07884757 -0.3462364 ], shape=(3,), dtype=float64)


### 2.3 元组

In [5]:
x = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
y = np.array(["cat", "dog", "fox"])

dataset1 = tf.data.Dataset.from_tensor_slices((x, y))
for item_x, item_y in dataset1:
    print(item_x.numpy())
    print("*"*5)
    print(item_y.numpy())
    print("="*10)

[1. 2.]
*****
b'cat'
[3. 4.]
*****
b'dog'
[5. 6.]
*****
b'fox'


### 2.4 字典

In [6]:
dataset1 = tf.data.Dataset.from_tensor_slices({"features": x, "labels": y})
for item in dataset1:
    print(item["features"])
    print("*"*5)
    print(item["labels"])
    print("="*10)

tf.Tensor([1. 2.], shape=(2,), dtype=float64)
*****
tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor([3. 4.], shape=(2,), dtype=float64)
*****
tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor([5. 6.], shape=(2,), dtype=float64)
*****
tf.Tensor(b'fox', shape=(), dtype=string)


### 2.5 混合

In [7]:
dataset1 = tf.data.Dataset.from_tensor_slices(({"features": x, "labels": y}, [1, 2, 3]))
for item1, item2 in dataset1:
    print(item1, "-------->", item2)

{'features': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([1., 2.])>, 'labels': <tf.Tensor: shape=(), dtype=string, numpy=b'cat'>} --------> tf.Tensor(1, shape=(), dtype=int32)
{'features': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([3., 4.])>, 'labels': <tf.Tensor: shape=(), dtype=string, numpy=b'dog'>} --------> tf.Tensor(2, shape=(), dtype=int32)
{'features': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([5., 6.])>, 'labels': <tf.Tensor: shape=(), dtype=string, numpy=b'fox'>} --------> tf.Tensor(3, shape=(), dtype=int32)


## 3. shuffle的使用
>作用是把`dataset`的数据打乱顺序

In [8]:
dataset1 = dataset1.shuffle(10000)
for item in dataset1:
    print(item)

({'features': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([1., 2.])>, 'labels': <tf.Tensor: shape=(), dtype=string, numpy=b'cat'>}, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
({'features': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([3., 4.])>, 'labels': <tf.Tensor: shape=(), dtype=string, numpy=b'dog'>}, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
({'features': <tf.Tensor: shape=(2,), dtype=float64, numpy=array([5., 6.])>, 'labels': <tf.Tensor: shape=(), dtype=string, numpy=b'fox'>}, <tf.Tensor: shape=(), dtype=int32, numpy=3>)


## 4. repeat  batch 的使用
> repeat 重复几分数据 <br/>
batch 每一个迭代出的元素包含几份数据 <br/>
如果repeat()没有传入数据，则重复无限次

In [9]:
dataset2 = tf.data.Dataset.from_tensor_slices(np.arange(5))
dataset2 = dataset2.repeat(3) # 重复3份数据
for item in dataset2:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)


In [10]:
dataset2 = dataset2.batch(4) # 一个元素快 4个数据样本
for item in dataset2:
    print(item)

tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)
tf.Tensor([4 0 1 2], shape=(4,), dtype=int64)
tf.Tensor([3 4 0 1], shape=(4,), dtype=int64)
tf.Tensor([2 3 4], shape=(3,), dtype=int64)


In [11]:
# 联合使用
# 1. 从内存分装一个dataset    from_tensor_slices
# 2. 打乱顺序   shuffle
# 3. 重复epoch，即数据复制几份  repeat
# 4. 定义元素块   batch

dataset3 = tf.data.Dataset.from_tensor_slices(np.arange(5))
dataset3 = dataset3.shuffle(10000)
dataset3 = dataset3.repeat(3).batch(4)

for item in dataset3:
    print(item)

tf.Tensor([0 4 3 1], shape=(4,), dtype=int64)
tf.Tensor([2 1 0 3], shape=(4,), dtype=int64)
tf.Tensor([4 2 1 4], shape=(4,), dtype=int64)
tf.Tensor([0 2 3], shape=(3,), dtype=int64)


## 5. take 从dataset取几个元素

In [12]:
dataset4 = tf.data.Dataset.from_tensor_slices(np.arange(10))

for item in dataset4.take(2):  # 取两个元素
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)


## 6. interleave
>作用是处理`dataset`的元素，并返回的数据，聚合成一个新的`dataset` <br/>
用例：第一的dataset分装数据文件名称，使用interleave 处理dataset里的每一个文件名，读取数据，所有文件数据返回一个新的dataset

In [13]:
dataset5 = tf.data.Dataset.from_tensor_slices(np.arange(9))
dataset5 = dataset5.repeat(2).batch(5)
for item in dataset5:
    print(item)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 0], shape=(5,), dtype=int64)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)
tf.Tensor([6 7 8], shape=(3,), dtype=int64)


In [14]:
dataset5 = dataset5.interleave(
    lambda ele: tf.data.Dataset.from_tensor_slices(ele),
    cycle_length=3,  # 3个并行取数据
    block_length=3  # 一次取3个数据
)

for item in dataset5:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)


## 7. map的用法
>map 不改变dataset元素的数量， 只改变元素的数据结构

In [15]:
dataset6 = tf.data.Dataset.from_tensor_slices(np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]))  

for item in dataset6:
    print(item)
print("*"*10)

def func(ele):
   # ele = ele +111
    return ele[:2], 11, ele[0]

dataset6 = dataset6.map(func)

for x, y, z in dataset6:
    print(x)
    print(y)
    print(z)
    print("="*10)


tf.Tensor([1. 2. 3.], shape=(3,), dtype=float64)
tf.Tensor([4. 5. 6.], shape=(3,), dtype=float64)
tf.Tensor([7. 8. 9.], shape=(3,), dtype=float64)
**********
tf.Tensor([1. 2.], shape=(2,), dtype=float64)
tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor(1.0, shape=(), dtype=float64)
tf.Tensor([4. 5.], shape=(2,), dtype=float64)
tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor(4.0, shape=(), dtype=float64)
tf.Tensor([7. 8.], shape=(2,), dtype=float64)
tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor(7.0, shape=(), dtype=float64)
